# Custom lightkurve extraction

Let's extract the lightcurve from the Sector 10 TPF so we know we can do it for Sector 36.

In [ ]:
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import lightkurve as lk

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
sr = lk.search_tesscut('Luhman 16')
sr

In [ ]:
tpf = sr.download(cutout_size=(16, 51))

Coordinates from Apai et al. 2021

In [ ]:
c_TESS = SkyCoord(ra=162.303282427*u.degree, 
             dec=-53.317573814*u.degree, 
             distance=1.0/0.50114 * u.pc, 
             pm_ra_cosdec=-2763*u.mas/u.yr,
             pm_dec=+358*u.mas/u.yr, 
             radial_velocity=20.0*u.km/u.s,
             frame='icrs',
             obstime=Time('2019.26', format='decimalyear')
            )

In [ ]:
x0_cutout, y0_cutout = tpf.wcs.world_to_pixel(c_TESS)
x0, y0 = tpf.column+x0_cutout, tpf.row+y0_cutout

In [ ]:
ny, nx = tpf.flux[0].shape

In [ ]:
xvec, yvec = np.arange(tpf.column, tpf.column+nx), np.arange(tpf.row, tpf.row+ny)

In [ ]:
xx, yy = np.meshgrid(xvec, yvec)

In [ ]:
radius_threshold = 1.1

In [ ]:
pixel_distance = np.sqrt((xx-x0)**2 + (yy-y0)**2)

In [ ]:
aperture_mask = pixel_distance < radius_threshold

In [ ]:
tpf.plot(aperture_mask=aperture_mask);

In [ ]:
def background_subtract_lightcurve(tpf, aperture_mask):
    '''background subtract a lightcurve'''
    median_image = np.nanmedian(tpf.flux.value,axis=0)
    lo = np.percentile(median_image, 15)
    bkg_pixel_mask = (median_image < lo) & ~aperture_mask
    
    n_bkg_pixels = bkg_pixel_mask.sum()
    n_tar_pixels = aperture_mask.sum()
    
    bkg_lc = tpf.to_lightcurve(aperture_mask=bkg_pixel_mask)
    tar_lc = tpf.to_lightcurve(aperture_mask=aperture_mask)
    net_lc = tar_lc - (bkg_lc / n_bkg_pixels * n_tar_pixels)
    
    return net_lc

In [ ]:
corrected_lc = background_subtract_lightcurve(tpf, aperture_mask).remove_outliers(sigma=3).normalize()

In [ ]:
corrected_lc.plot();

Woohoo!  Looks similar to Apai et al. 2021.  Let's compare it head-to-head.

In [ ]:
import glob

In [ ]:
fns = glob.glob('../data/TESS/**/*.fits', recursive=True)
fn = fns[0]

In [ ]:
lc_pathos = lk.LightCurve.read(fn, format='pathos').remove_outliers(sigma=3).normalize()

In [ ]:
ax=corrected_lc.plot(label='This Work (lightkurve)')
lc_pathos.plot(ax=ax, label='Apai et al. 2021 (pathos)')
ax.set_ylim(0.85)
plt.savefig('../figures/TESS_Sector_10_lightkurve_demo.png', dpi=300, bbox_inches='tight')

Great!  We get about the same answer.  Let's save our lightcurve to a file.

In [ ]:
aperture_mask.sum()

In [ ]:
corrected_lc.to_fits(path='../data/TESS/lightkurve_custom_4pixel.fits', aperture_mask=aperture_mask)